In [1]:
import os
import joblib # ou pickle

# --- BLOC DE CONFIGURATION DES CHEMINS (À METTRE AU DÉBUT) ---

# Le chemin vers le dossier 'models' est UN NIVEAU AU-DESSUS (../) du dossier 'notebooks'
MODELS_DIR = '../models'

# On s'assure que ce dossier existe. S'il n'existe pas, on le crée.
if not os.path.exists(MODELS_DIR):
    os.makedirs(MODELS_DIR)

# -----------------------------------------------------------------

In [2]:
import pandas as pd
import numpy as np
import random
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pickle

In [3]:
# --- Génération des données synthétiques ---
num_samples = 2000
data = []

for _ in range(num_samples):
    duree_anticipation_minutes = random.randint(5, 120)
    est_fin_semaine = random.choice([0, 1]) # 1 pour Vendredi, 0 sinon
    nb_hs_recentes_heures = random.randint(0, 15)
    charge_travail_jour = round(random.uniform(0.1, 1.0), 2)
    
    # Règle métier pour la décision réelle (vérité terrain)
    decision_reelle = 0 # 0 = Justification requise (par défaut)
    
    # On tolère si l'employé est parti un peu avant un vendredi ET qu'il a fait des HS récemment
    if duree_anticipation_minutes <= 30 and est_fin_semaine == 1 and nb_hs_recentes_heures > 5:
        decision_reelle = 1 # 1 = Tolérer
    # On tolère aussi les toutes petites sorties anticipées si la charge de travail est faible
    elif duree_anticipation_minutes <= 10 and charge_travail_jour < 0.4:
        decision_reelle = 1

    data.append([
        duree_anticipation_minutes,
        est_fin_semaine,
        nb_hs_recentes_heures,
        charge_travail_jour,
        decision_reelle
    ])

# Création du DataFrame
columns = [
    'duree_anticipation_minutes', 
    'est_fin_semaine', 
    'nb_hs_recentes_heures', 
    'charge_travail_jour', 
    'decision_reelle'
]
df_sortie = pd.DataFrame(data, columns=columns)

# Sauvegarde du dataset
df_sortie.to_csv('dataset_sortie_anticipee.csv', index=False)

print("Dataset généré et sauvegardé.")
df_sortie.head()

Dataset généré et sauvegardé.


,duree_anticipation_minutes,est_fin_semaine,nb_hs_recentes_heures,charge_travail_jour,decision_reelle
0,42,1,13,0.67,0
1,84,1,3,0.18,0
2,103,1,12,0.32,0
3,36,1,0,0.34,0
4,61,1,5,0.54,0


In [4]:
# --- Entraînement du modèle ---
features = [
    'duree_anticipation_minutes', 
    'est_fin_semaine', 
    'nb_hs_recentes_heures', 
    'charge_travail_jour'
]
target = 'decision_reelle'

X = df_sortie[features]
y = df_sortie[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Création et entraînement du modèle LightGBM
model_sortie = lgb.LGBMClassifier(random_state=42)
model_sortie.fit(X_train, y_train)

print("Modèle de sortie anticipée entraîné !")

# Évaluation
y_pred = model_sortie.predict(X_test)
print("\nRapport de classification :")
print(classification_report(y_test, y_pred))

[LightGBM] [Info] Number of positive: 150, number of negative: 1450
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000553 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 227
[LightGBM] [Info] Number of data points in the train set: 1600, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.093750 -> initscore=-2.268684
[LightGBM] [Info] Start training from score -2.268684
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

In [5]:
# --- Sauvegarde du modèle ---
model_filename = os.path.join(MODELS_DIR, 'model_sortie_anticipee.pkl')
with open(model_filename, 'wb') as file:
    pickle.dump(model_sortie, file)

print(f"Modèle sauvegardé dans '{model_filename}'")

Modèle sauvegardé dans '../models\model_sortie_anticipee.pkl'
